In [46]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [47]:
## Load the trained model, scaler picle, onehot

model = load_model('model.h5')



#Load encoder and scaler

with open('le.pkl','rb') as file:
    le = pickle.load(file)
    
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
    
    

In [48]:
input_data = { 'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [49]:
input_data['Geography']

'France'

In [50]:
geo_encoder = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
Geo_transformed = pd.DataFrame(geo_encoder, columns = onehot_encoder_geo.get_feature_names_out(['Geography']) )
Geo_transformed

C:\Users\vinay\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [51]:
df = pd.DataFrame([input_data])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [52]:
new_df = pd.concat([df.reset_index(drop = True),Geo_transformed], axis =1)
new_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [53]:
new_df['Gender'] = le.transform(new_df['Gender'])

In [54]:
new_df = new_df.drop('Geography', axis =1)
new_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [55]:
new_df1 = scaler.transform(new_df)

new_df1

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [56]:
prediction = model.predict(new_df1)
prediction

1/1 [==============================] - 0s 44ms/step


array([[0.03439034]], dtype=float32)

In [57]:
prediction_proba= prediction[0][0]
prediction_proba

0.03439034

In [58]:

if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
